In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
import seaborn as sns


data = pd.read_csv('E202-COMP7117-TD01-00 - clustering.csv')
data = data[['SpecialDay','VisitorType','Weekend','ProductRelated_Duration','ExitRates']]

for i in range(len(data)):
  if data['SpecialDay'][i] == 'HIGH':
    data['SpecialDay'][i] = 2
  elif data['SpecialDay'][i] == 'NORMAL':
    data['SpecialDay'][i] = 1
  elif data['SpecialDay'][i] == 'LOW':
    data['SpecialDay'][i] = 0


  if data['VisitorType'][i] == 'Returning_Visitor':
    data['VisitorType'][i] = 2
  elif data['VisitorType'][i] == 'New_Visitor':
    data['VisitorType'][i] = 1
  elif data['VisitorType'][i] == 'Other':
    data['VisitorType'][i] = 0


  if data['Weekend'][i] == 'TRUE':
    data['Weekend'][i] = 1
  elif data['Weekend'][i] == 'FALSE':
    data['Weekend'][i] = 0
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarn

In [3]:
  data

,SpecialDay,VisitorType,Weekend,ProductRelated_Duration,ExitRates
0,0,2,False,0,3
1,0,2,False,0,3
2,0,2,False,0,3
3,0,2,False,0,3
4,0,2,True,1,2
...,...,...,...,...,...
3627,0,0,False,0,3
3628,0,0,False,0,3
3629,0,0,False,3,2
3630,0,0,False,0,2


In [4]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data= scaler.fit_transform(data)

print(data)

pca = PCA(n_components=3)
pca = pca.fit(data)
dataset = pca.transform(data)
print(dataset)

[[0.         1.         0.         0.         1.        ]
 [0.         1.         0.         0.         1.        ]
 [0.         1.         0.         0.         1.        ]
 ...
 [0.         0.         0.         1.         0.66666667]
 [0.         0.         0.         0.         0.66666667]
 [0.         0.         0.         1.         1.        ]]
[[ 0.56815537  0.58899039 -0.25828465]
 [ 0.56815537  0.58899039 -0.25828465]
 [ 0.56815537  0.58899039 -0.25828465]
 ...
 [-0.2690612  -0.00289453 -0.33706206]
 [ 0.28498296  0.5383081  -0.36543525]
 [-0.06794636  0.08187724 -0.29525993]]


In [5]:
import tensorflow.compat.v1 as tf
from matplotlib import pyplot as plt

tf.disable_v2_behavior()


class SOM:


    def __init__(self, height, width, input_dimension):
        self.height = height
        self.width = width
        self.input_dimension = input_dimension
        self.location = [tf.to_float([y, x])
                         for y in range(height) for x in range(width)]

        self.weight = tf.Variable(tf.random_normal(
            [width*height, input_dimension]))
            
        self.input = tf.placeholder(tf.float32, [input_dimension])

        best_matching_unit = self.get_bmu()

        self.updated_weight, self.rate_stacked = self.update_neighbour(
            best_matching_unit)

    def get_bmu(self):
        square_difference = tf.square(self.input - self.weight)
        distance = tf.sqrt(tf.reduce_mean(square_difference, axis=1))

        bmu_index = tf.argmin(distance)
        bmu_location = tf.to_float(
            [tf.div(bmu_index, self.width), tf.mod(bmu_index, self.width)])

        return bmu_location

    def update_neighbour(self, bmu):
        learning_rate = 0.6

        sigma = tf.to_float(tf.maximum(self.width, self.height)/2)

        square_difference = tf.square(self.location - bmu)
        distance = tf.sqrt(tf.reduce_mean(square_difference, axis=1))

        neighbour_strength = tf.exp(
            tf.div(tf.negative(tf.square(distance)), 2 * tf.square(sigma)))

        rate = neighbour_strength * learning_rate
        total_node = self.width * self.height
        rate_stacked = tf.stack([tf.tile(tf.slice(rate, [i], [1]), [
                                self.input_dimension]) for i in range(total_node)])

        input_weight_difference = tf.subtract(self.input, self.weight)

        weight_difference = tf.multiply(rate_stacked, input_weight_difference)

        weight_new = tf.add(self.weight, weight_difference)

        return tf.assign(self.weight, weight_new), rate_stacked

    def train(self, dataset, num_of_epoch):
        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            sess.run(init)

            for i in range(num_of_epoch):
                for data in dataset:
                    sess.run(self.updated_weight, feed_dict={
                        self.input: data
                    })

            cluster = [[] for i in range(self.height)]
            location = sess.run(self.location)
            weight = sess.run(self.weight)

            for i, loc in enumerate(location):
                print(i, loc[0])
                cluster[int(loc[0])].append(weight[i])

            self.cluster = cluster


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:

def main():

    height = 6
    width = 6

    input_dimension = 3
    som = SOM(height, width, input_dimension)
    som.train(dataset, 5000)

    plt.imshow(som.cluster)
    plt.show()

main()


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
